In [ ]:
#@title: Installing and loading all the neccessary libraries
!pip3 install  BERTopic keybert emoji install tweet-preprocessor pyvis snscrape
import re
import pandas as pd
import seaborn as sns
import itertools
import plotly.express as px
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import itertools
import pandas as pd
import snscrape.modules.twitter as sntwitter
from multiprocessing import Pool

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 19.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#@title Scrape a total of 150k #ESG Twitters
import snscrape.modules.twitter as sntwitter
df = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper('#ESG since:2022-03-31 until:2023-03-31 lang:en').get_items(),150000))
tweet_df=df[['date','rawContent','hashtags','cashtags']]
tweet_df.to_csv('/content/drive/MyDrive/NLP/NDC/ESG_150K_new.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('/content/drive/MyDrive/NLP/NDC/ESG_150K_new.csv')
df=df.dropna(subset=['rawContent', 'date'])
tweet_df=df.drop_duplicates(subset=["rawContent"], keep=False).reset_index()
tweet_df.info()

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145482 entries, 0 to 145481
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   index       145482 non-null  int64 
 1   date        145482 non-null  object
 2   rawContent  145482 non-null  object
 3   hashtags    145456 non-null  object
 4   cashtags    5002 non-null    object
dtypes: int64(1), object(4)
memory usage: 5.5+ MB


In [ ]:
tweet_df

,index,date,rawContent,hashtags,cashtags
0,0,2023-03-30 23:57:03+00:00,New Climate News: \n\nCorporate Interests ‘Wat...,"['climatecrisis', 'climateaction', 'esg', 'car...",NaN
1,1,2023-03-30 23:55:53+00:00,"Hey @GOP, if you think that #ESG is going away...",['ESG'],NaN
2,2,2023-03-30 23:54:35+00:00,"At Seagate, we’re driven by a sustainable data...","['Sustainability', 'Inclusivity', 'Governance'...",NaN
3,3,2023-03-30 23:42:01+00:00,Read how AI and blockchain can create innovati...,"['ai', 'blockchain', 'ESG']",NaN
4,4,2023-03-30 23:36:09+00:00,#Externalities #SocialisingLosses #Privatising...,"['Externalities', 'SocialisingLosses', 'Privat...",NaN
...,...,...,...,...,...
145477,149995,2022-07-30 10:40:03+00:00,#InvestmentManagement firms are in a unique po...,"['InvestmentManagement', 'ESG']",NaN
145478,149996,2022-07-30 10:25:21+00:00,It is a moment of pride for all of us at @Hind...,"['HZLCares', 'Safety', 'ESG']",NaN
145479,149997,2022-07-30 10:19:26+00:00,on a mission @CIPD to understand how property ...,"['esg', 'listentolocals']",NaN
145480,149998,2022-07-30 10:09:54+00:00,Watch x4 videos over 20 mins that will ensure ...,"['ESG', 'greenwashing', 'PR', 'reputation', 't...",NaN


In [ ]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import matplotlib.pyplot as plt
import numpy as np
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Preprocess tweets
stopwords_list = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

# Function to clean and preprocess tweet text
def clean_tweet(tweet):
  tweet = re.sub(r"http\S+", "", tweet)  # Remove URLs
  tweet = re.sub(r"@\w+", "", tweet) # Remove mentions
  #tweet = re.sub(r"#\w+", "", tweet) # Remove hashtags
  tweet = re.sub(r"RT\s+", "", tweet) # Remove retweets
  tweet = re.sub(r"[^\w\s]", "", tweet) # Remove punctuation
  tweet = re.sub(r"\d+", "", tweet) # Remove numbers
  tweet = tweet.lower() # Convert to lowercase
  tweet = tweet.strip() # Remove Extraneous whitespace
  tokens = word_tokenize(tweet) # Tokenize tweet into words
  tokens = [token for token in tokens if token not in ['esg','amp','mcclelland']] ## Remove ESG and AMP 
  tokens = [lemmatizer.lemmatize(w) for w in tokens] ## Lemmatizing Words
  tweet = ' '.join(tokens) # Join words back into a sentence
  return tweet


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
## Cleaning Tweets
tweet_df['clean_text'] = tweet_df['rawContent'].apply(clean_tweet)

In [ ]:
tweet_df['clean_text']

0         new climate news corporate interest watered do...
1                hey if you think that is going away it not
2         at seagate were driven by a sustainable datasp...
3         read how ai and blockchain can create innovati...
4         externality socialisinglosses privatisingprofi...
                                ...                        
145477    investmentmanagement firm are in a unique posi...
145478    it is a moment of pride for all of u at a our ...
145479    on a mission to understand how property develo...
145480    watch x video over min that will ensure your e...
145481    apply the metaverse to your business and grow ...
Name: clean_text, Length: 145482, dtype: object

In [ ]:
docs=tweet_df.clean_text.tolist() # Converting Tweets to a list 
year=pd.to_datetime(tweet_df['date']).dt.year.tolist()
date=pd.to_datetime(tweet_df['date']).tolist()
#date
#topics, probs = topic_model.fit_transform(docs)

In [ ]:
#@title Setting up BERTopic Pipeline
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance

embedding_model = SentenceTransformer("all-MiniLM-L6-v2") # Step 1 - Extract embeddings
umap_model = UMAP(n_neighbors=100, n_components=5, min_dist=0.0, metric='cosine', low_memory=True, random_state=42) # Step 2 - Reduce dimensionality
hdbscan_model = HDBSCAN(min_cluster_size=200, metric='euclidean', cluster_selection_method='eom', prediction_data=True) # Step 3 - Cluster reduced embeddings
vectorizer_model = CountVectorizer(stop_words="english") # Step 4 - Tokenize topics
ctfidf_model = ClassTfidfTransformer() # Step 5 - Create topic representation
# Base Model
topic_model_final_200 = BERTopic(nr_topics='auto',
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  ctfidf_model=ctfidf_model)

In [ ]:
## Fitting Model
topics, probs = topic_model_final_200.fit_transform(docs)

In [ ]:
#topic_model_final_200.save("/content/drive/MyDrive/NLP/topic_model_final_200")
#topic_model_final_200 = BERTopic.load('/content/drive/MyDrive/NLP/topic_model_final_200')

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
## Frequency Table for Topics
topic_model_final_200.get_topic_info()

,Topic,Count,Name
0,-1,76969,-1_sustainability_company_business_read
1,0,10473,0_woke_blackrock_pension_retirement
2,1,5722,1_social_environmental_governance_csrfriday
3,2,4962,2_corpgov_board_governance_boardofdirectors
4,3,4440,3_globalwarming_climatecrisis_climatescam_clim...
5,4,3741,4_mining_lithium_ev_battery
6,5,3413,5_carbon_netzero_emission_zero
7,6,3098,6_investing_investment_fund_investor
8,7,2532,7_entrepreneur_podcast_listen_figure
9,8,2381,8_blockchain_crypto_bitcoin_web


In [ ]:
# Reduce outliers using the c-tf-idf strategy
new_topics = topic_model_final_200.reduce_outliers(docs, topics,strategy="c-tf-idf")
topic_model_final_200.update_topics(docs, topics=new_topics, vectorizer_model=vectorizer_model)

In [ ]:
## Plotting Dendrogram for topic
topic_model_final_200.visualize_hierarchy(color_threshold=1.5,width=800,height=200)

In [ ]:
#topic_model_final_200.get_topic_info()
#topic_model_final_200.save("/content/drive/MyDrive/NLP/topic_model_final_200_no_outlier")

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
# Plotting Similarity Matrix
topic_model_final_200.visualize_heatmap(height=800,width=800)

In [ ]:
## Plotting Topic Words for Topics 
topic_model_final_200.visualize_barchart(top_n_topics=12)

In [ ]:
#Relabeling Topics
topic_model_final_200.set_topic_labels({-1:"Unclassified",0:"Woke & Black Rock ",1:"ESG & Business Practises",2:"ESG Governance Pillar",
                              3:"Climate Action & Controvesy",4:"EV Batteries",5:"Net-Zero Target",6:"Investment Fund",
                              7:"Entreprenuer",8:"Blockchain & Bitcoin",9:"Fossil and RE Industry",
                              10:"Conflict of Interest",11:"Supply Chain",12:"Green Washing",
                              13:"Disclosure Reporting",14:"SDGs",15:"Recycling & Circular Econ",
                              16:"CSR & Business",17:"Real Estate ",18:"Women & Investment",
                              19:"ESG Analytics",20:"Cloud and Digital Tech.",21:"Electric Vehicles",
                              22:"Podcast",23:"Social Innovation & ETF",24:"Social Impact and Jobs",
                              25:"AI & ML",26:"Biodiversity",27:"FinTech & Startup",
                              28:"Insurance Industries",29:"Diversity Inclusion ",30:"Nuclear",
                              31:"Food & Agriculture",32:"Green Bonds",33:"ESG reporting & EU",
                              34:"Charity & Volunteering",35:"Shareholders",36:"Cybersecurity",37:"Tax Transparency",
                              38:"HR & Employment",39:"Financial Leadership",40:"Inflation Reduction Act",
                              41:"Risk Compliance & Management",42:"Deloitte & Mampa",43:"Green Finance",
                              44:"Safety & Blasting",45:"Moody Rating",46:"Webinar",47:"Music Industry",
                              48:"Health Care",49:"Venture Capital",50:"Fashion and Textile",
                              51:"Shapping & Maritime",52:"Responsible and Sustainable Invest.",
                              53:"IOT",54:"Climate Risk & Banking"})
topic_model_final_200.custom_labels_



['Unclassified',
 'Woke & Black Rock ',
 'ESG & Business Practises',
 'ESG Governance Pillar',
 'Climate Action & Controvesy',
 'EV Batteries',
 'Net-Zero Target',
 'Investment Fund',
 'Entreprenuer',
 'Blockchain & Bitcoin',
 'Fossil and RE Industry',
 'Conflict of Interest',
 'Supply Chain',
 'Green Washing',
 'Disclosure Reporting',
 'SDGs',
 'Recycling & Circular Econ',
 'CSR & Business',
 'Real Estate ',
 'Women & Investment',
 'ESG Analytics',
 'Cloud and Digital Tech.',
 'Electric Vehicles',
 'Podcast',
 'Social Innovation & ETF',
 'Social Impact and Jobs',
 'AI & ML',
 'Biodiversity',
 'FinTech & Startup',
 'Insurance Industries',
 'Diversity Inclusion ',
 'Nuclear',
 'Food & Agriculture',
 'Green Bonds',
 'ESG reporting & EU',
 'Charity & Volunteering',
 'Shareholders',
 'Cybersecurity',
 'Tax Transparency',
 'HR & Employment',
 'Financial Leadership',
 'Inflation Reduction Act',
 'Risk Compliance & Management',
 'Deloitte & Mampa',
 'Green Finance',
 'Safety & Blasting',
 'Mo

In [ ]:
### Dynamic Topic Modeling
topics_over_time = topic_model_final_200.topics_over_time(docs, date, nr_bins=8,global_tuning=True, evolution_tuning=True)
topic_model_final_200.visualize_topics_over_time(topics_over_time, top_n_topics=10,normalize_frequency=True)

In [1]:
!pip3 install -U datapane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.5/506.5 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━

In [3]:
import datapane as dp 
#from datapane import Report
dp.login(token="b243465657222386c1c29240639a607294ef3267bba0f287701c")

In [ ]:
## Setup graphs for Datapane
c1=topic_model_final_200.visualize_heatmap(height=1000,width=1000,custom_labels=True)
c2=topic_model_final_200.visualize_hierarchy(color_threshold=1.5,custom_labels=True)
c3=topic_model_final_200.visualize_barchart(top_n_topics=40,custom_labels=True)
c4=topic_model_final_200.visualize_topics_over_time(topics_over_time, top_n_topics=10,custom_labels=True,normalize_frequency=True,height=500)
#c4=topic_model_final_200.visualize_topics_over_time(topics_over_time, top_n_topics=10,normalize_frequency=True,normalize_frequency=True)
report = dp.View(dp.Blocks(dp.Plot(c1, caption='Similarity Heatmap of Topics'), dp.Plot(c2, caption='Hierarchical Cluster of Topics'), columns=2),dp.Plot(c3, caption='Top Word Score for Top 40 Topics'),dp.Plot(c4, caption='Dynamic Topic Modeling for Top 10 Topics'))

In [ ]:
dp.upload_report(report,name="BERTopic + ESG Tweets", open=True, visibility='PUBLIC')

Uploading report and associated data - *please wait...*

Report successfully uploaded - view and share at <a href='https://cloud.datapane.com/reports/9Ar8qZ7/bertopic-esg-tweets/' target='_blank'>here</a>.

Uploaded CloudReport - view at https://cloud.datapane.com/reports/9Ar8qZ7/bertopic-esg-tweets/

In [ ]:
topic_model_final_200.get_topic_info()

In [ ]:
topic_model_final_200.save("/content/drive/MyDrive/NLP/topic_model_final_200_final")
#topic_model_final_200_final = BERTopic.load('/content/drive/MyDrive/NLP/topic_model_final_200_final')